In [1]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack, clone_model

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [3]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
import pandas as pd
import numpy as np

In [131]:
query = """
select date(time), count(*) from point
where time > '2019-11-01'
group by 1
order by 1
"""
display(pd.read_sql_query(query, engine))

,date,count
0,2019-11-01,1
1,2019-11-02,2
2,2019-11-03,1
3,2019-11-04,1
4,2019-11-05,1
5,2019-11-06,2
6,2019-11-07,1
7,2019-11-09,3
8,2019-11-10,3
9,2019-11-11,1


# Find tracks for day

In [209]:
date = "2019-11-30"

In [210]:
bracket_query = """
select track.id, min(time) as min_time, max(time) as max_time from track
join point on track_id = track.id
group by track.id
having abs(date(min(time)) - %(date)s) < 2  or abs(date(max(time)) - %(date)s) < 2
"""
display(pd.read_sql_query(bracket_query, engine, params={'date': date}))


,id,min_time,max_time
0,12744,2019-12-01 05:11:29+00:00,2019-12-01 05:12:36+00:00
1,1522,2019-12-01 08:54:26+00:00,2019-12-01 08:54:26+00:00
2,12692,2019-11-30 00:49:21+00:00,2019-11-30 00:54:11+00:00
3,12686,2019-11-30 00:04:24+00:00,2019-11-30 00:06:49+00:00
4,12756,2019-12-01 10:20:40+00:00,2019-12-02 00:35:14+00:00
...,...,...,...
203,12675,2019-11-29 07:52:30+00:00,2019-11-29 07:59:21+00:00
204,1417,2019-11-30 00:54:22+00:00,2019-11-30 00:54:22+00:00
205,12712,2019-11-30 06:37:32+00:00,2019-11-30 07:12:21+00:00
206,12742,2019-12-01 04:52:27+00:00,2019-12-01 04:58:22+00:00


In [211]:
bracket_tracks = [session.query(GPSTrack).get(int(t)) 
                  for t in pd.read_sql_query(bracket_query, engine, params={'date': date})[['id']].values]
len(bracket_tracks)

208

In [212]:
day_tracks = [t for t in bracket_tracks 
              if str(t.start.localtime.date()) == date or str(t.end.localtime.date()) == date]
len(day_tracks)

103

In [213]:
set([t.filename for t in day_tracks])

{'2019-11-29.json.gz',
 '2019-11-30 2.json.gz',
 '2019-12-01.json.gz',
 None,
 'inreach.gpx'}

In [214]:
set([t.name for t in day_tracks])

{'1, Nachisan',
 '156-1, Nachisan',
 '2, Tsukiji 6-Chōme',
 '3006, Ichinono',
 '396, Nachisan',
 '447, Nachisan',
 '8, Nachisan',
 'Aneel Nazareth (9931878)',
 'Daimonzaka',
 'Hotel Kuu Kyoto (ホテル空)',
 'Kii-Katsuura Station (紀伊勝浦駅)',
 'Kumano Nachi Taisha (熊野那智大社)',
 'Kyoto Beer Lab (京都ビアラボ)',
 'Nachi Falls (那智の瀧)',
 'Nachisan',
 None,
 'RIO',
 'Seiganto-ji',
 'Shingū',
 'Shinkansen Kyoto Station (東海道新幹線 京都駅)',
 'Shinkansen Nagoya Station (東海道新幹線 名古屋駅)',
 'Three Story pagoda',
 'Unknown Place',
 'Veg Out',
 '勝浦駅 バス停',
 '南紀勝浦温泉 万清楼',
 '熊野交通 勝浦駅前出札所',
 '那智の滝前バス停'}

In [215]:
set([t.start.localtime.date() for t in day_tracks])

{datetime.date(2019, 11, 29), datetime.date(2019, 11, 30)}

# Visualization

In [216]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,
    Polyline, 
    Popup,
    GeoJSON,
    DrawControl
)
from ipywidgets import HTML

from traitlets import link

In [217]:
def marker(t):
    marker = Marker(location=t.points[0].lat_lon)
    marker.popup = HTML(f"{t.id}: {t.name}")
    return marker

def add_track(m, t, **kwargs):
        l = t.as_polyline(
            fill=False,
            **kwargs
        )
        m.add_layer(l)
        l.popup = HTML(f"{t.id}: {t.name or t.filename}<br/>{t.points[0].time} - {t.points[-1].time}")

In [218]:
b = list(day_tracks[0].bounds)
for t in day_tracks:
    tb = t.bounds
    b[0] = min(b[0], tb[0])
    b[1] = min(b[1], tb[1])
    b[2] = max(b[2], tb[2])
    b[3] = max(b[3], tb[3])
    
center = (b[1] + b[3]/2, (b[0] + b[2])/2)
b

[135.758244, 33.62674146537339, 136.884294, 35.369608]

In [219]:
m = Map(center=center, zoom=2, close_popup_on_click=False)

markers = []
for track in day_tracks:
    if len(track.points) == 1 and track.name is not None:
        markers.append(marker(track))
    else:
        add_track(m, track)
m.add_layer(MarkerCluster(markers=markers))

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Cleanup overlap

In [220]:
from intervaltree import IntervalTree
from datetime import timedelta

In [221]:
intervals = IntervalTree()
for t in day_tracks:
    start = t.start.time
    end = t.end.time
    if start == end:
        end += timedelta(seconds=1)
    intervals.addi(start, end, [t])
intervals.split_overlaps()
intervals.merge_equals(data_reducer=lambda a, b: a+b)

In [222]:
len(intervals)

153

In [223]:
for i in sorted(intervals):
    print(f"{str(i.begin)}\t{str(i.end)}\t{i.end - i.begin}\t{' '.join(set([t.name or t.filename for t in i.data]))}")

2019-11-29 07:59:25+00:00	2019-11-29 21:21:10+00:00	13:21:45	南紀勝浦温泉 万清楼
2019-11-29 21:21:10+00:00	2019-11-29 21:21:11+00:00	0:00:01	南紀勝浦温泉 万清楼
2019-11-29 21:21:11+00:00	2019-11-29 22:54:04+00:00	1:32:53	南紀勝浦温泉 万清楼
2019-11-29 22:54:27+00:00	2019-11-29 22:55:15+00:00	0:00:48	2019-11-30 2.json.gz
2019-11-29 22:55:15+00:00	2019-11-29 23:02:14+00:00	0:06:59	2019-11-30 2.json.gz Aneel Nazareth (9931878)
2019-11-29 23:02:14+00:00	2019-11-29 23:02:18+00:00	0:00:04	Aneel Nazareth (9931878)
2019-11-29 23:02:18+00:00	2019-11-29 23:02:19+00:00	0:00:01	Aneel Nazareth (9931878) Kii-Katsuura Station (紀伊勝浦駅)
2019-11-29 23:02:19+00:00	2019-11-29 23:04:46+00:00	0:02:27	Aneel Nazareth (9931878) Kii-Katsuura Station (紀伊勝浦駅)
2019-11-29 23:04:46+00:00	2019-11-29 23:04:47+00:00	0:00:01	Aneel Nazareth (9931878) Kii-Katsuura Station (紀伊勝浦駅)
2019-11-29 23:04:47+00:00	2019-11-29 23:20:59+00:00	0:16:12	Aneel Nazareth (9931878) Kii-Katsuura Station (紀伊勝浦駅)
2019-11-29 23:20:59+00:00	2019-11-29 23:21:00+00:00	0:00:0

# Output

In [236]:
import gpxpy
from geopy.distance import distance

In [256]:
def points_to_gpx_track(points, tracks):
    t = gpxpy.gpx.GPXTrack(
            name=f"{points[0].localtime.time()} {' '.join({t.type or '' for t in tracks})}",
            description="\t".join({t.description or "" for t in tracks}),
        )
    t.comment="\t".join({t.name or t.filename or "" for t in tracks})
    t.source="\t".join({t.source or "" for t in tracks})
    t.type="\t".join({t.type or "" for t in tracks})
    s = gpxpy.gpx.GPXTrackSegment()
    t.segments.append(s)
    for p in points:
        s.points.append(gpxpy.gpx.GPXTrackPoint(
            latitude=p.latitude, longitude=p.longitude, elevation=p.elevation, time=p.time))
    return t

In [258]:
gpx = gpxpy.gpx.GPX()
for i in sorted(intervals):
    track_ids = [t.id for t in i.data]
    points = (
        session.query(GPSPoint)
        .filter(GPSPoint.track_id.in_(track_ids))
        .filter(GPSPoint.time.between(i.begin, i.end))
        .distinct(GPSPoint.time)
        .order_by(GPSPoint.time)
    ).all()
    if not points:
        continue
    
    if len(points) == 1:
        p = points[0]
        t = p.track
        w = gpxpy.gpx.GPXWaypoint(
            latitude=p.latitude, longitude=p.longitude, elevation=p.elevation,
            time=p.time, name=t.name, description=t.description, type=t.type,
            comment=t.source or t.filename
        )
        gpx.waypoints.append(w)
    else:
        tracks = {p.track for p in points}
        if len(tracks) > 1:
            total_d = 0
            max_s = 0
            one_hour = timedelta(hours=1)
            for j in range(1, len(points)):
                d = distance((points[j-1].latitude, points[j-1].longitude),
                             (points[j].latitude, points[j].longitude)).miles
                total_d += d
                t = (points[j].time - points[j-1].time) / one_hour
                s = d/t
                # print(d, t, s)
                max_s = max(max_s, s)
            total_t = (points[-1].time - points[0].time) / one_hour
            avg_s = total_d / total_t
            print(f"max speed: {max_s}  avg_speed: {avg_s}")
            # speed is too fast, might be pingponging between two tracks, include both
            if max_s > 100:
                for track in tracks:
                    track_id = track.id
                    track_points = [p for p in points if p.track_id == track_id]
                    gpx.tracks.append(points_to_gpx_track(track_points, [track]))
                    print("filtered track")
                    gpx.tracks[-1].number = len(gpx.tracks)
        gpx.tracks.append(points_to_gpx_track(points, tracks))
        gpx.tracks[-1].number = len(gpx.tracks)
    print(f"{str(points[0].localtime)}\t{str(points[-1].localtime.time())}\t{i.end - i.begin}\t{len(points)}\t{set(t.name or t.filename for t in i.data)}")

    

max speed: 5.854965130803271  avg_speed: 0.037850041687130666
2019-11-29 16:59:25+09:00	06:21:10	13:21:45	211	{'南紀勝浦温泉 万清楼'}
2019-11-30 06:21:10+09:00	06:21:10	0:00:01	1	{'南紀勝浦温泉 万清楼'}
max speed: 89.31588175797907  avg_speed: 0.5921260527393218
2019-11-30 06:21:16+09:00	07:54:04	1:32:53	327	{'南紀勝浦温泉 万清楼'}
2019-11-30 07:54:27+09:00	07:54:27	0:00:48	1	{'2019-11-30 2.json.gz'}
max speed: 19.929364606756327  avg_speed: 2.9002463917958954
2019-11-30 07:55:15+09:00	08:02:14	0:06:59	129	{'2019-11-30 2.json.gz', 'Aneel Nazareth (9931878)'}
2019-11-30 08:02:18+09:00	08:02:18	0:00:01	1	{'Aneel Nazareth (9931878)', 'Kii-Katsuura Station (紀伊勝浦駅)'}
max speed: 2.1493404427365213  avg_speed: 0.1550711208734164
2019-11-30 08:02:20+09:00	08:04:46	0:02:27	31	{'Aneel Nazareth (9931878)', 'Kii-Katsuura Station (紀伊勝浦駅)'}
max speed: 17.345712971111  avg_speed: 17.345712971111
2019-11-30 08:04:46+09:00	08:04:47	0:00:01	2	{'Aneel Nazareth (9931878)', 'Kii-Katsuura Station (紀伊勝浦駅)'}
max speed: 10.3477717937549

In [259]:
with open(f"{date}.gpx", "w") as fh:
    print(gpx.to_xml(version="1.1"), file=fh)